# Distributed Pytorch training using Horovod via OCI Jobs

## Contents

1. [Background](#Background)
1. [Prerequisites](#Prerequisites)
1. [Train](#Train)
1. [Setup IAM](#Setup%20IAM)
1. [Build](#Build)

---

## Background

Horovod is a distributed deep learning training framework for TensorFlow, Keras, PyTorch, and MXNet. This notebook example shows how to use Horovod with Tensorflow in OCI Data Science Jobs .

For more information about the Horovod with TensorFlow , please visit [Horovod-Tensorflow](https://horovod.readthedocs.io/en/stable/tensorflow.html)

---

## Prerequisites

### 1. Install ads package >= 2.5.9

In [ ]:
!pip3 install oracle-ads

### 2. Install docker:

https://docs.docker.com/get-docker

## Train
### Training script
A sample training script 'sample.py' which will be used as by different workers in the setup for distributeed training

This script uses Horovod framework for distributed training where Horovod-related lines are commented starting with `Horovod:`. For example, `Horovod: add Horovod DistributedOptimizer`, `Horovod: initialize optimize` and etc.

In [ ]:
%%writefile sample.py

import argparse
import os
from filelock import FileLock

import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
import torch.utils.data.distributed
import horovod.torch as hvd

# Training settings
parser = argparse.ArgumentParser(description='PyTorch MNIST Example')
parser.add_argument('--batch-size', type=int, default=64, metavar='N',
                    help='input batch size for training (default: 64)')
parser.add_argument('--test-batch-size', type=int, default=1000, metavar='N',
                    help='input batch size for testing (default: 1000)')
parser.add_argument('--epochs', type=int, default=10, metavar='N',
                    help='number of epochs to train (default: 10)')
parser.add_argument('--lr', type=float, default=0.01, metavar='LR',
                    help='learning rate (default: 0.01)')
parser.add_argument('--momentum', type=float, default=0.5, metavar='M',
                    help='SGD momentum (default: 0.5)')
parser.add_argument('--no-cuda', action='store_true', default=False,
                    help='disables CUDA training')
parser.add_argument('--seed', type=int, default=42, metavar='S',
                    help='random seed (default: 42)')
parser.add_argument('--log-interval', type=int, default=10, metavar='N',
                    help='how many batches to wait before logging training status')
parser.add_argument('--fp16-allreduce', action='store_true', default=False,
                    help='use fp16 compression during allreduce')
parser.add_argument('--use-adasum', action='store_true', default=False,
                    help='use adasum algorithm to do reduction')
parser.add_argument('--data-dir',
                    help='location of the training dataset in the local filesystem (will be downloaded if needed)')

args = parser.parse_args()
args.cuda = not args.no_cuda and torch.cuda.is_available()

# Horovod: initialize library.
hvd.init()
torch.manual_seed(args.seed)

if args.cuda:
    # Horovod: pin GPU to local rank.
    torch.cuda.set_device(hvd.local_rank())
    torch.cuda.manual_seed(args.seed)


# Horovod: limit # of CPU threads to be used per worker.
torch.set_num_threads(1)

kwargs = {'num_workers': 1, 'pin_memory': True} if args.cuda else {}
data_dir = args.data_dir or './data'
with FileLock(os.path.expanduser("~/.horovod_lock")):
    train_dataset = \
        datasets.MNIST(data_dir, train=True, download=True,
                       transform=transforms.Compose([
                           transforms.ToTensor(),
                           transforms.Normalize((0.1307,), (0.3081,))
                       ]))
# Horovod: use DistributedSampler to partition the training data.
train_sampler = torch.utils.data.distributed.DistributedSampler(
    train_dataset, num_replicas=hvd.size(), rank=hvd.rank())
train_loader = torch.utils.data.DataLoader(
    train_dataset, batch_size=args.batch_size, sampler=train_sampler, **kwargs)

test_dataset = \
    datasets.MNIST(data_dir, train=False, transform=transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.1307,), (0.3081,))
    ]))
# Horovod: use DistributedSampler to partition the test data.
test_sampler = torch.utils.data.distributed.DistributedSampler(
    test_dataset, num_replicas=hvd.size(), rank=hvd.rank())
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=args.test_batch_size,
                                          sampler=test_sampler, **kwargs)


class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        self.conv2_drop = nn.Dropout2d()
        self.fc1 = nn.Linear(320, 50)
        self.fc2 = nn.Linear(50, 10)

    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
        x = x.view(-1, 320)
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training=self.training)
        x = self.fc2(x)
        return F.log_softmax(x)


model = Net()

# By default, Adasum doesn't need scaling up learning rate.
lr_scaler = hvd.size() if not args.use_adasum else 1

if args.cuda:
    # Move model to GPU.
    model.cuda()
    # If using GPU Adasum allreduce, scale learning rate by local_size.
    if args.use_adasum and hvd.nccl_built():
        lr_scaler = hvd.local_size()

# Horovod: scale learning rate by lr_scaler.
optimizer = optim.SGD(model.parameters(), lr=args.lr * lr_scaler,
                      momentum=args.momentum)

# Horovod: (optional) compression algorithm.
compression = hvd.Compression.fp16 if args.fp16_allreduce else hvd.Compression.none


def metric_average(val, name):
    tensor = torch.tensor(val)
    avg_tensor = hvd.allreduce(tensor, name=name)
    return avg_tensor.item()


@hvd.elastic.run
def train(state):
    # post synchronization event (worker added, worker removed) init ...
    for state.epoch in range(state.epoch, args.epochs + 1):
        state.model.train()

        train_sampler.set_epoch(state.epoch)
        steps_remaining = len(train_loader) - state.batch

        for state.batch, (data, target) in enumerate(train_loader):
            if state.batch >= steps_remaining:
                break

            if args.cuda:
                data, target = data.cuda(), target.cuda()
            state.optimizer.zero_grad()
            output = state.model(data)
            loss = F.nll_loss(output, target)
            loss.backward()
            state.optimizer.step()
            if state.batch % args.log_interval == 0:
                # Horovod: use train_sampler to determine the number of examples in
                # this worker's partition.
                print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                    state.epoch, state.batch * len(data), len(train_sampler),
                    100.0 * state.batch / len(train_loader), loss.item()))
            state.commit()
        state.batch = 0


def test():
    model.eval()
    test_loss = 0.
    test_accuracy = 0.
    for data, target in test_loader:
        if args.cuda:
            data, target = data.cuda(), target.cuda()
        output = model(data)
        # sum up batch loss
        test_loss += F.nll_loss(output, target, size_average=False).item()
        # get the index of the max log-probability
        pred = output.data.max(1, keepdim=True)[1]
        test_accuracy += pred.eq(target.data.view_as(pred)).cpu().float().sum()

    # Horovod: use test_sampler to determine the number of examples in
    # this worker's partition.
    test_loss /= len(test_sampler)
    test_accuracy /= len(test_sampler)

    # Horovod: average metric values across workers.
    test_loss = metric_average(test_loss, 'avg_loss')
    test_accuracy = metric_average(test_accuracy, 'avg_accuracy')

    # Horovod: print output only on first rank.
    if hvd.rank() == 0:
        print('\nTest set: Average loss: {:.4f}, Accuracy: {:.2f}%\n'.format(
            test_loss, 100. * test_accuracy))


# Horovod: wrap optimizer with DistributedOptimizer.
optimizer = hvd.DistributedOptimizer(optimizer,
                                     named_parameters=model.named_parameters(),
                                     compression=compression,
                                     op=hvd.Adasum if args.use_adasum else hvd.Average)


# adjust learning rate on reset
def on_state_reset():
    for param_group in optimizer.param_groups:
        param_group['lr'] = args.lr * hvd.size()


state = hvd.elastic.TorchState(model, optimizer, epoch=1, batch=0)
state.register_reset_callbacks([on_state_reset])
train(state)
test()

## Setup IAM

### Create the Dynamic Group
```
ALL {resource.type = ‘datasciencejobrun’, resource.compartment.id = <COMPARTMENT_OCID>}
```

### Create the following Policy
```
Allow dynamic-group <DYNAMIC_GROUP_NAME> to use log-content in compartment <COMPARTMENT_NAME>
Allow dynamic-group <DYNAMIC_GROUP_NAME> to use log-groups in compartment <COMPARTMENT_NAME>
Allow dynamic-group <DYNAMIC_GROUP_NAME> to inspect repos in compartment <COMPARTMENT_NAME>
Allow dynamic-group <DYNAMIC_GROUP_NAME> to inspect vcns in compartment <COMPARTMENT_NAME>
Allow dynamic-group <DYNAMIC_GROUP_NAME> to manage objects in compartment <COMPARTMENT_NAME> where any {target.bucket.name=<BUCKET_NAME>}
Allow dynamic-group <DYNAMIC_GROUP_NAME> to manage buckets in compartment <COMPARTMENT_NAME> where any {target.bucket.name='BUCKET_NAME'}
```

## Build

### Initialize a distributed-training folder
By this time, you would have created a training file (or files) - sample.py from the above example. Now running the command below

In [ ]:
!ads opctl distributed-training init --framework horovod --version v1

### Build Docker image

The sample code is assumed to be in the current working directory.

In [ ]:
!docker build -f oci_dist_training_artifacts/horovod/docker/pytorch.cpu.Dockerfile -t (IMAGE-NAME):(TAG) .

### Push the Docker Image to OCIR

#### Steps
1. Follow the instructions to setup container registry from [here](https://docs.oracle.com/en-us/iaas/Content/Registry/Tasks/registrypushingimagesusingthedockercli.htm)
2. Make sure you create a repository in OCIR to push the image
3. Tag Local Docker image that needs to be pushed -> `docker tag "image-identifier" "target-tag"`
4. Push the Docker image from the client machine to Container Registry -> `docker push "target-tag"`

##### Tag Docker image

In [ ]:
!docker tag hvdjob-cpu-pytorch-1.0 iad.ocir.io/<TENANCY_NAME>/horovod:hvdjob-cpu-pytorch-1.0

##### Push Docker Image

In [ ]:
!docker push iad.ocir.io/<TENANCY_NAME>/horovod:hvdjob-cpu-pytorch-1.0

### Define your workload yaml:

The yaml file is a declarative way to express the workload.
Edit the `"oci_dist_training_artifacts/horovod/jobrun_config/jobrun_config_hvd_tf.yaml"` file to specify run config

The following variables are tenancy specific that needs to be modified

| Variable | Description |
| :-------- | :----------- |
|compartmentId|OCID of the compartment where Data Science projects are created|
|projectId|OCID of the project created in Data Science service|
|subnetId|OCID of the subnet attached your Job|
|logGroupId|OCID of the log group for JobRun logs|
|image|Image from OCIR to be used for JobRuns|
|workDir|URL to the working directory for opctl|
|WORKSPACE|Workspace with the working directory to be used|
|entryPoint|The script to be executed when launching the container|

```yaml
kind: distributed
apiVersion: v1.0
spec:
  infrastructure: # This section maps to Job definition. Does not include environment variables
    kind: infrastructure
    type: dataScienceJob
    apiVersion: v1.0
    spec:
      projectId: <PROJECT_OCID>
      compartmentId: <COMPARTMENT_OCID>
      displayName: <DISPLAY_NAME>
      logGroupId: <LOG_GROUP_OCID>
      subnetId: <SUBNET_OCID>
      shapeName: <COMPUTE_SHAPE>
      blockStorageSize: <SIZE_IN_GB>
      blockStorageSizeInGBs: <SIZE_IN_GB>
  cluster:
    kind: HOROVOD
    apiVersion: v1.0
    spec:
      image: "<REGION_CODE>.ocir.io/<TENANCY_NAME>/<REPOSITORY_NAME>:<IMAGE_TAG>"
      workDir:  "oci://<BUCKET_NAME>@<NAMESPACE>/"
      name: "<DISPLAY_NAME>"
      config:
        env:
          - name: MIN_NP
            value: 2
          - name: MAX_NP
            value: 8
          - name: SLOTS
            value: 2
          - name: WORKER_PORT
            value: 12345
          - name: GLOO_TIMEOUT_SECONDS
            value: 90
          - name: START_TIMEOUT
            value: 700
          - name: ENABLE_TIMELINE
            value: 1
          - name: SYNC_ARTIFACTS
            value: 1
          - name: WORKSPACE
            value: "horovod-ws"
          - name: WORKSPACE_PREFIX
            value: "hvd"
      main:
        name: "scheduler"
        replicas: 1
      worker:
        name: "worker"
        replicas: 2
  runtime:
    kind: python
    apiVersion: v1.0
    spec:
      entryPoint: "/code/sample.py"
      args: ""
      kwargs: ""
      env:
        - name: ARTIFACTS_DIR
          value: "/opt/ml"
```